In [2]:
import openai
from qdrant_client import QdrantClient


### EMBEDDING function

In [3]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        model=model,
        input=text
    )
    return response.data[0].embedding


### RETREVAL function

In [49]:
def retrieve_data(query, qdrant_client, k=5):
    """
    Retrieve k most similar items to the query from Qdrant collection.
    """
    response = qdrant_client.query_points(
        collection_name="amazon_items-collection-00",
        query=get_embeddings(query),
        limit=k
    )

    retrieved_context_ids = [];
    # this is description of the product
    retrieved_context = [];
    similarity_scores = [];
    for point in response.points:   
        retrieved_context_ids.append(point.payload["parent_asin"])
        retrieved_context.append(point.payload["description"])
        similarity_scores.append(point.score)
    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores
    }



In [ ]:
retrieve_data("what various tablets are available", k=10)

### Format retrieved context function (above data works as context for LLM, need to format that for LLM input)

In [10]:
def format_retrieved_context(context_data):
    formatted_context = ""
    for id, context in zip(context_data["retrieved_context_ids"], context_data["retrieved_context"]):
            formatted_context += f"- {id}: {context}\n"
    return formatted_context

In [12]:
print(format_retrieved_context(retrieve_data("what various tablets are available", k=10)))

- B09QGNB537: ESR for iPad Air 5th Generation Case 2022 / iPad Air 4th Case 2020, Rugged Protection, Detachable Magnetic Cover, Adjustable Portrait/Landscape Stand with Raised Screen View, 8 Stand Angles, Black Compatibility: only compatible with iPad Air 5th Generation 2022 and iPad Air 4th Generation 2020; not compatible with any other iPad models Detachable Magnetic Cover: keep your screen covered, fold it up into a stand for viewing/writing/portrait modes, or remove it for better ergonomics while gaming Rugged Protection: rugged shock-resistant polymer provides robust protection against scratches and drops Stable and Adjustable Stand: magnetically locks to 8 different stand positions that stay stable in bed, on the couch, or anywhere else Stand Your Way: raised screen view, portrait mode, and magnetic fridge mounting let you comfortably view and use your iPad in the office, kitchen, and beyond Complete Customer Support: detailed setup videos and FAQs, comprehensive 12-month warrant

### Create prompt

In [16]:
def create_prompt(query, preprocessed_retrieved_context):
    processed_context = format_retrieved_context(preprocessed_retrieved_context)
    
    prompt = f"""
    You are a helpful shopping assistant who can answer questions about the products in stock.
    You are given a question and a list of products with their descriptions.
    Your job is to answer the question based on the context.

    Instructions:
    - Answer the question based on the context only.
    - Never use the word "context" in your answer and refer to it as available products.
    - If you don't know the answer, say "I don't know".
    - If the question is not related to the context, say "I don't know".

    Question: 
    {query}
    Context: 
    {processed_context}
    """
    return prompt

In [32]:
prompt = create_prompt("what various tablets are available", retrieve_data("what various tablets are available", k=10))

### Use prompt to generate answer

In [ ]:
def generate_answer(prompt):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5
    )
    return response.choices[0].message.content

In [39]:
q = "what kinds of earphones are available?"
prompt = create_prompt(q, retrieve_data(q, k=10))
print(generate_answer(prompt))

There are several kinds of earphones available:

1. Kids Headphones (TUNEAKE) – Over-ear wired headphones with volume limited to 94dB, foldable, adjustable, no microphone, with 3.5mm jack, designed for kids' comfort and safety.

2. Open Ear Bluetooth Headphones – Wireless earbuds with open-ear design, Bluetooth 5.3, IPX7 waterproof, 60 hours playtime, earhooks for secure fit, allowing awareness of surroundings.

3. TELSOR Wireless Earbuds – Bluetooth 5.1 earbuds with touch control, noise cancelling mic, 30 hours playtime, IPX7 waterproof, ergonomic design, suitable for calls and music.

4. Wireless Bluetooth Earbuds (Sport S23-vine) – Bluetooth 5.3 earbuds with microphone, 37 hours playback, deep bass, IPX7 waterproof, ultra-light with charging case and smart touch.

5. Bone Conduction Headphones (Siniffo X15) – Wireless Bluetooth 5.3 open-ear sports earphones resting on cheekbones, IP56 waterproof, 8-hour battery, great for workouts and outdoor activities.

6. RUSAM GA33 A Game Earbud

### RAG pipeline - Combined

In [50]:
def rag_pipeline(query, top_k=5):
    qdrant_client = QdrantClient(url="http://localhost:6333")
    preprocessed_retrieved_context = retrieve_data(query, qdrant_client, top_k)
    prompt = create_prompt(query, preprocessed_retrieved_context)
    answer = generate_answer(prompt)
    return answer

In [51]:
print(rag_pipeline("what various tablets are available", top_k=10))

The available tablets compatible with accessories in the current stock are:

- iPad Air 5th Generation (2022)
- iPad Air 4th Generation (2020)
- iPad Pro 11" (1st, 2nd, 3rd, and 4th generation)
- iPad Pro 12.9" (3rd, 4th, 5th, and 6th generation)
- iPad Mini 6

There is a protective case suitable specifically for iPad Air 4th and 5th generations and a stylus pen compatible with iPad mini 6, iPad Air 4 and 5, and various iPad Pro models as listed above.

No other tablets are directly mentioned in the available products.
